In [1]:
from IPython.display import Image

将单词与向量相关联还有另一种常用的强大方法，就是使用密集的词向量（word vector），也叫词嵌入（word embedding）。  
one-hot 编码得到的向量是二进制的、稀疏的（绝大部分元素都 是 0）、维度很高的（维度大小等于词表中的单词个数），而词嵌入是低维的浮点数向量（即密 集向量，与稀疏向量相对），参见下图。与 one-hot 编码得到的词向量不同，词嵌入是从数据中 学习得到的。常见的词向量维度是 256、512 或 1024（处理非常大的词表时）。与此相对，onehot 编码的词向量维度通常为 20 000 或更高（对应包含 20 000 个标记的词表）。因此，词向量可 以将更多的信息塞入更低的维度中。  
![figure](img/wordvec.PNG)

获取词嵌入有两种方法。   
 在完成主任务（比如文档分类或情感预测）的同时学习词嵌入。在这种情况下，一开始 是随机的词向量，然后对这些词向量进行学习，其学习方式与学习神经网络的权重相同  
 在不同于待解决问题的机器学习任务上预计算好词嵌入，然后将其加载到模型中。这些 词嵌入叫作预训练词嵌入（pretrained word embedding）。

# 利用 Embedding 层学习词嵌入

要将一个词与一个密集向量相关联，最简单的方法就是随机选择向量。  
这种方法的问题在于， 得到的嵌入空间没有任何结构。例如，accurate 和 exact 两个词的嵌入可能完全不同，尽管它们 在大多数句子里都是可以互换的   

说得更抽象一点，词向量之间的几何关系应该表示这些词之间的语义关系。词嵌入的作用 应该是将人类的语言映射到几何空间中。例如，在一个合理的嵌入空间中，同义词应该被嵌入 到相似的词向量中，一般来说，任意两个词向量之间的几何距离（比如 L2 距离）应该和这两个 词的语义距离有关（表示不同事物的词被嵌入到相隔很远的点，而相关的词则更加靠近）。除了 距离，你可能还希望嵌入空间中的特定方向也是有意义的。为了更清楚地说明这一点，我们来 看一个具体示例  
下图中，四个词被嵌入在二维平面上，这四个词分别是 cat（猫）、dog（狗）、wolf（狼）
和 tiger（虎）。对于我们这里选择的向量表示，这些词之间的某些语义关系可以被编码为几何 变换。例如，从 cat 到 tiger 的向量与从 dog 到 wolf的向量相等，这个向量可以被解释为“从宠 物到野生动物”向量。同样，从 dog 到 cat 的向量与从 wolf到 tiger 的向量也相等，它可以被解 释为“从犬科到猫科”向量  
![figure2](img/vec2.PNG)  

在真实的词嵌入空间中，常见的有意义的几何变换的例子包括“性别”向量和“复数”向量。  
例如，将 king（国王）向量加上 female（女性）向量，得到的是 queen（女王）向量。将 king（国王） 向量加上 plural（复数）向量，得到的是 kings 向量。词嵌入空间通常具有几千个这种可解释的、 并且可能很有用的向量  

有没有一个理想的词嵌入空间，可以完美地映射人类语言，并可用于所有自然语言处理任 务？可能有，但我们尚未发现。此外，也不存在人类语言（human language）这种东西。世界上有许多种不同的语言，而且它们不是同构的，因为语言是特定文化和特定环境的反射。但从更 实际的角度来说，一个好的词嵌入空间在很大程度上取决于你的任务。英语电影评论情感分析 模型的完美词嵌入空间，可能不同于英语法律文档分类模型的完美词嵌入空间，因为某些语义 关系的重要性因任务而异。  

因此，合理的做法是对每个新任务都学习一个新的嵌入空间。幸运的是，反向传播让这种 学习变得很简单，而 Keras 使其变得更简单。我们要做的就是学习一个层的权重，这个层就是 Embedding 层

# 将一个 Embedding 层实例化

In [7]:
from keras.layers import Embedding
# Embedding 层至少需要两个参数： 标记的个数（这里是 1000，即最 大单词索引 +1）和嵌入的维度(这 里是 64)
embedding_layer = Embedding(1000, 64)

Using TensorFlow backend.


最好将 Embedding 层理解为一个字典，将整数索引（表示特定单词）映射为密集向量。它 接收整数作为输入，并在内部字典中查找这些整数，然后返回相关联的向量。Embedding 层实 际上是一种字典查找（见下图）。  
![vec3](img/vec3.PNG)

Embedding 层的输入是一个二维整数张量，其形状为 (samples, sequence_length)，  
每个元素是一个整数序列。它能够嵌入长度可变的序列，例如，对于前一个例子中的 Embedding 层，你可以输入形状为 (32, 10)（32 个长度为 10 的序列组成的批量）或 (64, 15)（64 个长度为 15 的序列组成的批量）的批量。不过一批数据中的所有序列必须具有相同的 长度（因为需要将它们打包成一个张量），所以较短的序列应该用 0 填充，较长的序列应该被截断。  
这个 Embedding 层返回一个形状为 (samples, sequence_length, embedding_ dimensionality) 的三维浮点数张量。然后可以用RNN层或一维卷积层来处理这个三维张量.   
将一个 Embedding 层实例化时，它的权重（即标记向量的内部字典）最开始是随机的，与
其他层一样。在训练过程中，利用反向传播来逐渐调节这些词向量，改变空间结构以便下游模 型可以利用。一旦训练完成，嵌入空间将会展示大量结构，这种结构专门针对训练模型所要解 决的问题  

我们将这个想法应用于你熟悉的 IMDB 电影评论情感预测任务。首先，我们需要快速准备 8
数据。将电影评论限制为前 10 000 个最常见的单词（第一次处理这个数据集时就是这么做的）， 然后将评论长度限制为只有 20 个单词。对于这 10 000 个单词，网络将对每个词都学习一个 8 维嵌入，将输入的整数序列（二维整数张量）转换为嵌入序列（三维浮点数张量），然后将这个 张量展平为二维，最后在上面训练一个 Dense 层用于分类。

## 加载 IMDB 数据，准备用于 Embedding 层

In [11]:
from keras.datasets import imdb
from keras import preprocessing 
max_features = 10000 
maxlen = 20
(x_train, y_train), (x_test, y_test) = imdb.load_data( num_words=max_features)

# 将整数列表转换成形状为(samples, maxlen) 的二维整数张量
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen) 
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

  163840/17464789 [..............................] - ETA: 8:25:10

TimeoutError: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。

## 在 IMDB 数据上使用 Embedding 层和分类器

In [ ]:
from keras.models import Sequential 
from keras.layers import Flatten, Dense, Embedding

In [ ]:
model = Sequential()
# 指定 Embedding 层的最大输入长度，以 便后面将嵌入输入展平。
model.add(Embedding(10000, 8, input_length=maxlen))# Embedding 层 激活的形状为(samples, maxlen, 8)

model.add(Flatten())# 将三维的嵌入张量展平成形状为 (samples, maxlen * 8) 的二维张量
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) 
model.summary()
history = model.fit(x_train, 
                    y_train, 
                    epochs=10, 
                    batch_size=32,
                    validation_split=0.2)

得到的验证精度约为 76%，考虑到仅查看每条评论的前 20 个单词，这个结果还是相当不错的。  
但请注意，仅仅将嵌入序列展开并在上面训练一个 Dense 层，会导致模型对输入序列中的 每个单词单独处理，而没有考虑单词之间的关系和句子结构（举个例子，这个模型可能会将 this movie is a bomb和 this movie is the bomb两条都归为负面评论）。更好的做法是在嵌入序列上添 加循环层或一维卷积层，将每个序列作为整体来学习特征。这也是接下来几节的重点。

## 使用预训练的词嵌入